In [6]:
# import libraries
import cv2
import os
from PIL import Image
import datetime
import numpy as np
import datetime
import numpy as np
import math

In [2]:
# SIFT initialization
detector = cv2.xfeatures2d.SIFT_create()
FLANN_INDEX_KDITREE = 0
flannParam = dict(algorithm=FLANN_INDEX_KDITREE, tree=5)
flann = cv2.FlannBasedMatcher(flannParam, {})
trainImg = cv2.imread("sift_img/dni5.jpg", 0)
trainKP, trainDesc = detector.detectAndCompute(trainImg, None)
MIN_MATCH_COUNT = 20

In [8]:
# 1 Getting files in multiple paths
# ruta = 'D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\escaneos_concursos'
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\data\\escaneos\\marzo_compras_2018'
rutas_personas  = [arch.name for arch in os.scandir(ruta) if arch.is_dir()]
total = 0

files_lista = []

for persona in rutas_personas:
    bolsas  = [arch.name for arch in os.scandir(ruta + "\\" + persona) if arch.is_dir()]
    for bolsa in bolsas:
        files = [arch.name for arch in os.scandir(ruta + "\\" + persona + "\\" + bolsa) if arch.is_file()]
        for file in files:
            files_lista.append(ruta + "\\" + persona + "\\" + bolsa + "\\" + file)
print("Terminó. {}".format(len(files_lista)))

Terminó. 170175


In [3]:
# 1 Getting files from inputh
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input'
files = [arch.name for arch in os.scandir(ruta) if arch.is_file()]
files_lista = []

for i in files:
    files_lista.append(ruta+'\\'+i)

print("Terminó. {}".format(len(files_lista)))

Terminó. 94


In [4]:
files_lista[:50]

['C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164448_00043.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164452_00148.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164452_00149.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164454_00245.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164458_00359.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164503_00537.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164503_00546.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164507_00664.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_0202072019164509_00749.jpg',
 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\MarzoCompras_020207201

In [5]:
# 1 Extracción del area de forma estática
y_min = 120
y_max = 213
x_min = 0
x_max = 833

falses = 0

#array's
h_lista = []
w_lista = []
p_lista = []
c_x_lista = []
c_y_lista = []
file_lista=[]
paths_lista=[]

print("Inicio {}".format(datetime.datetime.now()))
for filepath in files_lista:
    # Initializing a boolean variable with False
    dni_founded = False
    # Preprocessing section
    # print('Read: {}'.format(filepath))
    
    # cv2.imshow('image',image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    image = cv2.imread(filepath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sift_roi = gray[y_min:y_max,x_min:x_max]
    sift_roi = cv2.adaptiveThreshold(sift_roi,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,11)
    
    # SIFT DETECTION
    queryKP, queryDesc = detector.detectAndCompute(sift_roi, None)
    matches = flann.knnMatch(queryDesc, trainDesc, k=2)
    detection = False

    goodMatch = []
    for m, n in matches:
        if(m.distance < 0.75 * n.distance):
            goodMatch.append(m)
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp = []
        qp = []
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp, qp = np.float32((tp, qp))
        H, status = cv2.findHomography(tp, qp, cv2.RANSAC, 3.0)
        h, w = trainImg.shape
        trainBorder = np.float32([[[0,0], [0,h-1], [w-1,h-1], [w-1,0]]])
        queryBorder = cv2.perspectiveTransform(trainBorder, H)
        cv2.polylines(sift_roi, [np.int32(queryBorder)], True, (0, 255, 0), 3)
        dni_founded = True

    if dni_founded:
        #-------Comparacion de Hmax y Wmax-----
        # restringinedo valores a 0
        queryBorder = np.where(queryBorder<0,0,queryBorder)
        
        # Coordenadas "query"
        x0 = queryBorder[[[0],0]][[0],0]
        y0 = queryBorder[[[0],0]][[0],1]

        x1 = queryBorder[[[0],1]][[0],0]
        y1 = queryBorder[[[0],1]][[0],1]

        x2 = queryBorder[[[0],2]][[0],0]
        y2 = queryBorder[[[0],2]][[0],1]

        x3 = queryBorder[[[0],3]][[0],0]
        y3 = queryBorder[[[0],3]][[0],1]
        
        arr_x = np.array([x0,x1,x2,x3])
        x_maximo = arr_x.max()
        x_minimo = arr_x.min()
        c_x = int(arr_x.sum()/arr_x.size)

        arr_y =np.array([y0,y1,y2,y3])
        y_maximo = arr_y.max()
        y_minimo = arr_y.min()
        c_y = int(arr_y.sum()/arr_y.size)
        
        h_lista.append(y_maximo - y_minimo)
        w_lista.append(x_maximo - x_minimo)
        p_lista.append([(x0,y0),(x1,y1),(x2,y2),(x3,y3)])
        c_x_lista.append(c_x)
        c_y_lista.append(c_y)
        paths_lista.append(filepath)
    else:
        falses += 1
        h_lista.append(9999)
        w_lista.append(9999)
        p_lista.append([(0,0,),(0,0,),(0,0,),(0,0,)])
        c_x_lista.append(9999)
        c_y_lista.append(9999)
        paths_lista.append(filepath)
    # cv2.imshow("Sift_Roi", sift_roi)
    # cv2.imshow("Gray", gray)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
print("Terminó :D. Fin:{}. Total:{}".format(datetime.datetime.now(),falses))

# convirtiendo a nunmpy arrays
n_widths = np.array(w_lista)
n_heights = np.array(h_lista)
n_points = np.array(p_lista)
n_cxs = np.array(c_x_lista)
n_cys = np.array(c_y_lista)
n_paths = np.array(paths_lista)

# Saving nunmpy arrays
np.savetxt('areas_result/widths.txt', n_widths)
np.savetxt('areas_result/heights.txt', n_heights)
np.save('areas_result/points', n_points)
np.savetxt('areas_result/cxs.txt', n_cxs)
np.savetxt('areas_result/cys.txt', n_cys)
np.save('areas_result/paths', n_paths)

Inicio 2019-09-13 15:50:25.678528
Terminó :D. Fin:2019-09-13 15:50:30.111533. Total:0


In [2]:
import numpy as np
# convirtiendo a nunmpy arrays
n_widths = np.genfromtxt('areas_result/widths.txt')
n_heights = np.genfromtxt('areas_result/heights.txt')

n_cxs = np.genfromtxt('areas_result/cxs.txt')
n_cys = np.genfromtxt('areas_result/cys.txt')
n_paths = np.array('areas_result/paths.npy')

In [5]:
(np.argwhere(n_widths==9999) == np.argwhere(n_heights==9999)).sum()

0

In [3]:
def dni_rect_from_ce(x, y, cv2_v, img):
    w = img.shape[1]
    h = img.shape[0]
    
    color = (0, 0, 255)
    thicknes = 2
    
    fields = []
    res_dict = {}
    
    for key in traslation_dict.keys():
        tras_vector = np.array([traslation_dict[key]['tras_x'], traslation_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([traslation_dict[key]['width'], traslation_dict[key]['height']])
        
        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]
        
        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)
        # print('{} {}'.format(point,point_2))
    return res_dict['dni'][0], res_dict['dni'][1], res_dict['tel'][0], res_dict['tel'][1]

In [4]:
# Loading muestra numpy arrays
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\areas_result\\'

n_widths = np.genfromtxt(ruta+'widths.txt')
n_heights = np.genfromtxt(ruta+'heights.txt')
n_cxs = np.genfromtxt(ruta+'cxs.txt')
n_cys = np.genfromtxt(ruta+'cys.txt')
n_paths = np.load(ruta+'paths.npy')

In [14]:
# ------------------------------------------------------
traslation_dict_camioneta = {
    'nomb_ape1':{'tras_x':210,'tras_y':-140,'width':540,'height':60},
    'nomb_ape2':{'tras_x':-40,'tras_y':-82,'width':820,'height':50},
    'dni':{'tras_x':27,'tras_y':-40,'width':310,'height':55},
    'tel':{'tras_x':27+400,'tras_y':-40,'width':290,'height':55},
    'dir1':{'tras_x':100,'tras_y':7,'width':670,'height':50},
    'dir2':{'tras_x':-40,'tras_y':57,'width':820,'height':50},
    'distrito':{'tras_x':70,'tras_y':102,'width':800,'height':50},
    'mail':{'tras_x':60,'tras_y':152,'width':800,'height':50},
    'cc':{'tras_x':200,'tras_y':198,'width':800,'height':50}
}

traslation_dict = {
    'dni':{'tras_x':27,'tras_y':-44,'width':340,'height':75},
    'tel':{'tras_x':27+455,'tras_y':-44,'width':290,'height':75},
    'mail':{'tras_x':60,'tras_y':185,'width':800,'height':65},
    'nomb_ape1':{'tras_x':240,'tras_y':-180,'width':540,'height':80},
    'nomb_ape2':{'tras_x':-40,'tras_y':-105,'width':820,'height':65},
    'dir1':{'tras_x':100,'tras_y':20,'width':670,'height':65},
    'dir2':{'tras_x':-40,'tras_y':72,'width':820,'height':65},
    'distrito':{'tras_x':70,'tras_y':127,'width':800,'height':65},
    'cc':{'tras_x':200,'tras_y':240,'width':800,'height':65}
}


# traslation_dict = traslation_dict_camioneta.copy()

# 1 Visualizacion
y_min = 120
y_max = 213
x_min = 0
x_max = 833

print('Empezó en:{}'.format(datetime.datetime.now()))

# Calculando el arreglo de arhivos correctos
# checked_files = np.delete(n_paths, (np.where(n_paths==failed_files))[1])
# sample = np.take(checked_files,np.random.randint(0,checked_files.size,200))

# for i in range(failed_files.size):
for i in range(25):
# for i in range(n_paths.size):
    # Grabiing the file
    '''
    filename = failed_files[i][0]
    c_x = int(failed_cxs[i][0])
    c_y = int(failed_cys[i][0])
    '''
    # filename = checked_files[i]
    filename = n_paths[i]
    name = filename.split('\\')
    name = name[len(name)-1]
    name = name.split('.')[0]
    
    c_x = int(n_cxs[i])
    c_y = int(n_cys[i])
    
    wi = n_widths[i]
    hi = n_heights[i]
    
    x_minimo = int(c_x-(wi/2))
    x_maximo = int(c_x+(wi/2))
    y_minimo = int(c_y-(hi/2))
    y_maximo = int(c_y+(hi/2))
    
    image = cv2.imread(filename)
    frame = image.copy()
    
    found = False
    
    if (c_x==9999 or c_y==9999):
        cv2.putText(image,'{}'.format('Area no detectada'), (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        # print('{},{}'.format(filename,'no detectado'))
        found = False
    else:
        found = True
        
        print('{}'.format(filename))
        cv2.rectangle(image,(x_minimo,y_minimo+int(y_min)),(x_maximo,y_maximo+int(y_min)),(0,0,255),1)
        dni1, dni2, cel1, cel2 = dni_rect_from_ce(c_x, c_y+y_min, cv2, image)
        
        dniGray = cv2.cvtColor(frame[dni1[1]:dni2[1],dni1[0]:dni2[0]], cv2.COLOR_BGR2GRAY)
        dniGrayTresh = cv2.threshold(dniGray, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
        dniGrayBlur = cv2.medianBlur(dniGrayTresh, 1)
        
        celGray = cv2.cvtColor(frame[cel1[1]:cel2[1],cel1[0]:cel2[0]], cv2.COLOR_BGR2GRAY)
        celGrayTresh = cv2.threshold(celGray, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
        celGrayBlur = cv2.medianBlur(celGrayTresh, 1)
        
        # Find DNI and celular section
        # ocr_str = pytesseract.image_to_string(dniGray, lang='eng', config='--psm 6 --oem 2 -c tessedit_char_whitelist=0123456789')
        # ocr_str = pytesseract.image_to_string(gray, lang='spa', config='--psm 1 --oem 0')
        # ocr_str = ocr_str.replace("\n","").replace(" ","")
        # ocr_str = pytesseract.image_to_string(t_im, lang='spa', config='--psm 11 --oem 0 -c tessedit_char_whitelist=DNIdniTeléfonTELÉFON')
        # boxes = pytesseract.image_to_string(image, lang='spa_old', config='--psm 7 --oem 0 -c tessedit_char_whitelist=DNIdni')

        # Show section
        # print('{};{};{};{}'.format(file,ocr_str,'DNI' in ocr_str, 'Teléfono' in ocr_str))
        # t.append((str(file),str(ocr_str),'DNI' in ocr_str,'Teléfono' in ocr_str,))
        # cv2.putText(image,'({})'.format(ocr_str), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    cv2.imshow("Img", image)
    
    if found:
        # cv2.imshow("dniGray", dniGray)
        # cv2.imshow("dniGrayTresh", dniGrayTresh)
        cv2.imshow("dniGrayBlur", dniGrayBlur)
        cv2.imshow("ccelGrayBlur", celGrayBlur)
        # cv2.imwrite('areas/{}_dni.jpg'.format(name),dniGrayBlur)
        # cv2.imwrite('areas/{}_cel.jpg'.format(name),celGrayBlur)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
print("Terminó :D. Fin:{}".format(datetime.datetime.now()))

Empezó en:2019-09-27 17:35:20.844667
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164447_00007.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164447_00026.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164451_00135.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164452_00153.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164456_00293.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164457_00313.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164458_00356.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164459_00387.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164500_00429.jpg
C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164503_00544.jpg
C:\git\cuponesWong\CuponesWong\notebooks_fl